In [ ]:
# Name: Laijun Xu

**Part I**

In [1]:
#1&2.

import pandas as pd
import pymc as pm
import numpy as np

file_path = 'Walmart_sales.csv' 
data = pd.read_csv(file_path)


X = data['Temperature'].values.reshape(-1, 1) 
y = data['Holiday_Flag'].values  

with pm.Model() as logistic_model:

    betas = pm.Normal('betas', mu=0, sigma=10, shape=X.shape[1])  
    intercept = pm.Normal('intercept', mu=0, sigma=10)  

    logits = intercept + pm.math.dot(X, betas)
    
    p = pm.Deterministic('p', pm.math.sigmoid(logits))
    
    y_obs = pm.Bernoulli('y_obs', p=p, observed=y)
    
    trace = pm.sample(500, tune=200, target_accept=0.95)

summary = pm.summary(trace)
print(summary)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [betas, intercept]


Sampling 4 chains for 200 tune and 500 draw iterations (800 + 2_000 draws total) took 14 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
There were 1000 divergences after tuning. Increase `target_accept` or reparameterize.


            mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
betas[0]   0.369  0.403  -0.035    0.816      0.200    0.153       5.0   
intercept -0.514  0.307  -0.975   -0.136      0.143    0.109       5.0   
p[0]       0.552  0.448   0.098    1.000      0.222    0.170       5.0   
p[1]       0.558  0.442   0.109    1.000      0.219    0.168       5.0   
p[2]       0.556  0.444   0.105    1.000      0.220    0.169       5.0   
...          ...    ...     ...      ...        ...      ...       ...   
p[6430]    0.527  0.473   0.050    1.000      0.235    0.180       5.0   
p[6431]    0.527  0.473   0.050    1.000      0.235    0.180       5.0   
p[6432]    0.536  0.464   0.069    1.000      0.230    0.176       5.0   
p[6433]    0.534  0.466   0.065    1.000      0.231    0.177       5.0   
p[6434]    0.532  0.468   0.060    1.000      0.232    0.178       5.0   

           ess_tail  r_hat  
betas[0]      303.0   3.39  
intercept     328.0   2.29  
p[0]          178.0   2.

3. 
When choosing priors for a Bayesian model, it is essential to reflect on the level of information available about the parameters in question. Without strong prior information, it is common to opt for weakly informative or non-informative priors, which aim to impose minimal influence on the posterior estimates, allowing the data to speak for themselves.

For the 'betas' parameter, which represents the effect of the temperature on the log odds of the outcome (holiday shopping behavior in this case), a normal distribution with a large variance can be considered a weakly informative prior. For example, a 'Normal(0, 10^2)' distribution for 'betas' suggests we believe a priori that most of the mass of the distribution of this effect size lies within about two standard deviations of zero but does not rule out larger values.

For the 'intercept' parameter, which represents the baseline log odds of the outcome when the temperature is at zero, a similar approach can be taken. A 'Normal(0, 10^2)' prior for the 'intercept' would reflect a belief that the baseline probability is as likely to be high as it is to be low, without strong evidence in either direction.

In summary, the choice of 'Normal(0, 10^2)' for both 'betas' and 'intercept' represents a weakly informative prior that allows the data to largely determine the posterior while still regularizing the estimates to avoid extreme values that are not supported by the data.

**Part II**

For ridge regression, the penalized loss function can be written as:
$\text{Ridge Loss} = \frac{1}{2} \sum_{i=1}^n (y_i - x_i^T \beta)^2 + \lambda \sum_{j=1}^p \beta_j^2$

Under the Bayesian framework with normal priors on $\beta$ coefficients, we have:
$p(\beta | y, X) \propto \exp \left( -\frac{1}{2\sigma^2} \sum_{i=1}^n (y_i - x_i^T \beta)^2 \right) \exp \left( -\frac{1}{2\tau^2} \sum_{j=1}^p \beta_j^2 \right)$

Setting $\sigma = 1$ and $\lambda = \frac{1}{2\tau^2}$, we have:
$p(\beta | y, X) \propto \exp \left( -\frac{1}{2} \sum_{i=1}^n (y_i - x_i^T \beta)^2 - \lambda \sum_{j=1}^p \beta_j^2 \right)$

For lasso regression, the penalized loss function is given by:
$\text{Lasso Loss} = \frac{1}{2} \sum_{i=1}^n (y_i - x_i^T \beta)^2 + \lambda \sum_{j=1}^p |\beta_j|$

In Bayesian terms with Laplace priors on $\beta$ coefficients:
$p(\beta | y, X) \propto \exp \left( -\frac{1}{2\sigma^2} \sum_{i=1}^n (y_i - x_i^T \beta)^2 \right) \exp \left( -\frac{\lambda}{\sigma} \sum_{j=1}^p |\beta_j| \right)$

Setting $\sigma = 1$ and using the same $\lambda$ as in lasso, we have:
$p(\beta | y, X) \propto \exp \left( -\frac{1}{2} \sum_{i=1}^n (y_i - x_i^T \beta)^2 - \lambda \sum_{j=1}^p |\beta_j| \right)$

The statement to be written and understood is: "Bayesians do not optimize posterior distributions, they sample from them; but, the posterior distributions are nonetheless 'regularizations' of the likelihood through the prior."


**Part III**

In [1]:
import pymc as pm
import numpy as np
import pandas as pd

file_path = 'Walmart_sales.csv' 
data = pd.read_csv(file_path)

X = data['Temperature'].values[:, None]  # Predictor
y = data['Holiday_Flag'].values  # Response

# Robust regression model
with pm.Model() as robust_model:

    alpha = pm.Gamma('alpha', alpha=0.5, beta=0.5)
    beta = pm.Normal('beta', mu=0, sigma=10)
    intercept = pm.Normal('intercept', mu=0, sigma=10)
    
    lambda_ = pm.InverseGamma('lambda_', alpha=alpha, beta=alpha)

    y_pred = intercept + beta * X.flatten()
    likelihood = pm.StudentT('y', nu=1, mu=y_pred, lam=lambda_, observed=y)

    trace = pm.sample(1000, return_inferencedata=True)

# Identify outliers based on the posterior of lambda_i
lambda_posterior = trace.posterior['lambda_'].data.flatten()
outlier = np.where(lambda_posterior < np.percentile(lambda_posterior, 1))[0]

print('Outlier indices:', outlier)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta, intercept, lambda_]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 29 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
There were 2261 divergences after tuning. Increase `target_accept` or reparameterize.


Outlier indices: [2106 2107 2108 2109 2110 2111 2112 2113 2114 2115 2117]
